In [1]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy 

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re


In [6]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [69]:
tokens = tokenizer.encode("that great",return_tensors='pt')


In [70]:
tokens

tensor([[  101, 10203, 11838,   102]])

In [13]:
result = model(tokens)

In [71]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5089, -1.5103, -0.4764,  0.8670,  2.0642]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [14]:
result.logits

tensor([[-1.5089, -1.5103, -0.4764,  0.8670,  2.0642]],
       grad_fn=<AddmmBackward0>)

In [15]:
int(torch.argmax(result.logits))+1

5

In [19]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup=BeautifulSoup(r.text,'html.parser')
regex=re.compile(".*comment.*")
results=soup.find_all('p',{'class':regex})
reviews=[result.text for result in results]

In [33]:
results[0].text


'Great staff and food. \xa0Must try is the pan fried Gnocchi! \xa0The staff were really friendly and the coffee was good as well'

In [34]:
import pandas as pd
import numpy as np

In [39]:
df= pd.DataFrame(np.array(reviews),columns=['review'])

In [40]:
df

,review
0,Great staff and food. Must try is the pan fri...
1,It was ok. The coffee wasn't the best but it w...
2,I came to Social brew cafe for brunch while ex...
3,Ricotta hot cakes! These were so yummy. I ate ...
4,I went here a little while ago- a beautiful mo...
5,We came for brunch twice in our week-long visi...
6,Ron & Jo are on the go down under and Wow! We...
7,Great coffee and vibe. That's all you need. C...
8,Great coffee and vibe. That's all you need. C...
9,Good coffee and toasts. Straight up and down -...


In [54]:
df['review'].iloc[1]

"It was ok. The coffee wasn't the best but it was fine. The relish on the breakfast roll was yum which did make it sing. So perhaps I just got a bad coffee but the food was good on my visit."

In [48]:
def sentiment_score(review):
    tokens = tokenizer.encode(review,return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1
    

In [55]:
sentiment_score(df['review'].iloc[1])

3

In [56]:
df['sentiment']=df['review'].apply(lambda x: sentiment_score(x[:512]))

In [59]:
df

,review,sentiment
0,Great staff and food. Must try is the pan fri...,5
1,It was ok. The coffee wasn't the best but it w...,3
2,I came to Social brew cafe for brunch while ex...,5
3,Ricotta hot cakes! These were so yummy. I ate ...,5
4,I went here a little while ago- a beautiful mo...,2
5,We came for brunch twice in our week-long visi...,4
6,Ron & Jo are on the go down under and Wow! We...,5
7,Great coffee and vibe. That's all you need. C...,5
8,Great coffee and vibe. That's all you need. C...,4
9,Good coffee and toasts. Straight up and down -...,5
